# Using ClosedLoop Sagemaker Marketplace Models

This notebook will walk you through using a ClosedLoop model that has been published to the AWS Sagemaker Marketplace. A ClosedLoop account is not required 

To view a list of all of the models available on the SageMaker Marketplace please see the following link.

https://aws.amazon.com/marketplace/seller-profile?id=4134f86f-d5eb-4591-8634-e88e27eff5a6


## Install

Make sure you have the sagemaker API library installed https://github.com/aws/sagemaker-python-sdk#installing-the-sagemaker-python-sdk

`pip install sagemaker`


## Notes about the models input and output data.

All of the model packages published take the same input format and emit the same output format. You can find out more about what exactly the model is predicting from the AWS marketplace listing for that model package.

Data represented in a tabular format where columns and rows are represented as JSON. Each JSON has the id associated with it and optionally records from other fields defined in schema. Data type for the fields is listed in the schema.

Parameters:

schema – An ordered list of the columns in data.
dataSnapshot – The version of raw data used to generate the JSON. See getDataSnaphots for more details.
data – The actual data as a list of JSON.
idFields – Optional specification for list of ids to retrieve. If unspecified, data for all the population is returned.

You can find more detailed documentation at the following link https://docs.closedloop.ai/source/closedloop.api.objects.html#closedloop.api.services.JsonDataFrame

There is an example JsonDataFrame included with this notebook that contains a single record in the format the model is expecting.

## Set up the environment

In [ ]:
import boto3
import sagemaker as sage
from sagemaker import get_execution_role

In [ ]:
sagemaker_session = sage.session.Session()

In [ ]:
role = get_execution_role(session)

## Create Model

Using one of the model packages to create a model. You will need the ARN of your subscribed model package. You can get the ARN from the AWS console under Amazon SageMaker -> Model packages: -> My Subscriptions. 

Below is an example ARN for the asthma admissions model package.

In [ ]:
modelpackage_arn = "arn:aws:sagemaker:us-east-1:865070037744:model-package/closedloop-asthma-admissions-r-5fdd32ce46e75c940075636cef92fc2e"

In [ ]:
from sagemaker import ModelPackage

model = ModelPackage(role=role,
                     model_package_arn=modelpackage_arn,
                     sagemaker_session=sagemaker_session)

## Batch Inference

The model can be used for batch inference. The input data to the model is the exact same as for live inference, however it supports a Json Lines format where each line of the file represents a JsonDataFrame that will be submitted for predictions.

## Live Inference Endpoint

Now we demonstrate the creation of an endpoint for live inference. This uses the /invocations endpoint provided by SageMaker.

### Create endpoint

In [ ]:
inference = model.deploy(1, "ml.t2.medium")

In [ ]:
inference

### Submit data and get a prediction.

There is a sample input file located alongside this notebook containing the sample JsonDataFrame the model is expecting as input. To run this on your own data you will need to format your data in a similar manner.

In [ ]:
input_data = open('data/input/sagemaker-input.json').read()

In [ ]:
inference.predict(input_data)

### Cleanup Endpoint

In [ ]:
inference.delete_endpoint()